In [1]:
########## Multilabel - Data Augumented - Universal sentense Encoder -  Keras Modelling ####################

In [2]:
#################################################################

In [127]:
###################################################

import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Layer, concatenate, Concatenate, Reshape, Conv2D, Conv1D
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import backend as K

session  = tf.Session()
K.set_session(session)
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())



In [128]:
###################################################

from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = datasets.load_files(container_path = 'dataset_small_less_categories_v2',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])
    df.reset_index(inplace=True, drop=True)

In [129]:
df

,label,text
0,5,I want to travel back to my native place.
1,5,Want curfew pass for case of emergency travel.
2,5,I had started towards my home 120 km away on my car.
3,5,Firstline workers facing issues as no cabs are available
4,5,Want to go back to my family place. Currently staying in the hotel.
...,...,...
333,4,Economic downfall will soon create job crises.
334,4,Employee are not happy with the relief funds provided.
335,4,emi should be reduced and less interest rate should be charged on loans and other policies.
336,4,tv news channel focus more on fake news.


In [130]:
#trainset.data[0], trainset.target[0]

In [131]:
trainset

{'data': ["I want to travel back to my native place.\r\nWant curfew pass for case of emergency travel.\r\nI had started towards my home 120 km away on my car.\r\nFirstline workers facing issues as no cabs are available\r\nWant to go back to my family place. Currently staying in the hotel.\r\nWant to travel from bangalore to delhi. Any means of transport available?\r\nI haven't seen my family for a long time and they need my support. I need to be there.\r\nResiding in green zone but not able to get travel permssion even for getting essential household items.\r\nFew days back my vehicle was seized by police, now don't have any means of transport.\r\nMy visa about to get expire. Want some travel arrangements to be done to fly back to Austria.\r\nFive construction workers started walking from kerela to bihar but are stranded in karnataka border.\r\nI live in pg where its difficult to get food. When are flights and trains getting started.\r\nYou are giving permission to celebrities like Ris

In [132]:
pd.set_option('display.max_colwidth', 200)
df

,label,text
0,5,I want to travel back to my native place.
1,5,Want curfew pass for case of emergency travel.
2,5,I had started towards my home 120 km away on my car.
3,5,Firstline workers facing issues as no cabs are available
4,5,Want to go back to my family place. Currently staying in the hotel.
...,...,...
333,4,Economic downfall will soon create job crises.
334,4,Employee are not happy with the relief funds provided.
335,4,emi should be reduced and less interest rate should be charged on loans and other policies.
336,4,tv news channel focus more on fake news.


In [133]:
###################################################

import random
data=[]
length=len(df)

for i in range(length * 10):
    num1 = i % length
    num2 = num1
    while (num1 == num2):
        num2=random.randrange(length)
    data.append([df.label[num1],df.label[num2],df.text[num1] + ' ' + df.text[num2]])
    
for i in range(length):
    data.append([df['label'][i], df['label'][i], df['text'][i]])     
df2 = pd.DataFrame(data, columns=['label1','label2', 'text'])
#df2.append(pd.DataFrame([df['label'], df['label'], df['text']]))

In [134]:
df2['label2'].value_counts()

3    777
2    720
0    705
5    607
4    496
1    413
Name: label2, dtype: int64

In [135]:
trainset.target_names

['communication', 'essential_items', 'food', 'healthcare', 'others', 'travel']

In [136]:
df2.tail(10)

,label1,label2,text
3708,4,4,She is an absolute arrogant disgrace - shameful and no respect for the gentleman she was interviewing. She planned to gang up on him and hit him hard - lack of rational sense was from her. Pur...
3709,4,4,Think there's a minority who have an issue with NHS staff as I got verbally abused in a store - a woman shouted in my face as she pushed me to go in a queue. When I brought my ID out to get disco...
3710,4,4,Took me years to realise. It isn't ok! If the Warning signs show get out from abuse!
3711,4,4,They lost their jobs due to COVID-19. Request you to pls help them.
3712,4,4,I lost my job due to this sudden covid outbreak.
3713,4,4,Economic downfall will soon create job crises.
3714,4,4,Employee are not happy with the relief funds provided.
3715,4,4,emi should be reduced and less interest rate should be charged on loans and other policies.
3716,4,4,tv news channel focus more on fake news.
3717,4,4,work from home should be manditory for all companies


In [137]:
# df2.to_csv('dataset_multi.csv')

In [138]:
###################################################
import spacy
from nltk.stem import WordNetLemmatizer

In [277]:
###################################################
nlp = spacy.load("en_core_web_lg")
filter_POS = set(['NOUN', 'ADJ', 'VERB', 'PART'])
lemmatizer = WordNetLemmatizer()
# ADV, NOUN, ADP, PRON, SCONJ, PROPN, DET, SYM, INTJ
# PUNCT, NUM, AUX, X, CONJ, ADJ, VERB, PART, SPACE, CCONJ
# def filter_query(query):
#     query = query.lower()
#     doc = nlp(query)
#     tokens = [t.text for t in doc if t.pos_ in filter_POS]
#     return ' '.join(tokens)
gc = 0
def filter_query(query):
    query = query.lower()
    query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
    # print (query)
    doc = nlp(query)
    if len(doc) > 6:
        tokens = [t.text for t in doc if t.pos_ in filter_POS]
    else:
        tokens = [t.text for t in doc]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    filt_q = ' '.join(tokens)
    filt_q = re.sub(r'\b(n\'t|nt)\b', 'not', filt_q)
    filt_q = re.sub(r'\'ll\b', 'will', filt_q)
    return filt_q

In [140]:
df2['text'].shape

(3718,)

In [141]:
###################################################
df2['text_'] = df2['text']
df2['text'] = df2['text'].apply(filter_query)

# df['text_'] = df['text']
# df['text'] = df['text'].apply(filter_query)

In [142]:
# df2_ = df2.copy()

In [143]:
df2

,label1,label2,text,text_
0,5,2,want to travel native place can bread butter,I want to travel back to my native place. Where can I get bread and butter.
1,5,1,want curfew pas case emergency travel much needed essential supply not available nearby store,Want curfew pass for case of emergency travel. Much needed essential supplies not available in nearby store.
2,5,5,started home 120 km car update normal international flight will resume unpredictability covid-19 other country immigration change,I had started towards my home 120 km away on my car. Hi unfortunately we have no update on when normal international flights will resume due to the unpredictability of COVID-19 and other countries...
3,5,4,firstline worker facing issue cab available work home should manditory company,Firstline workers facing issues as no cabs are available work from home should be manditory for all companies
4,5,5,want to go family place staying hotel stuck 8 month old infant ca not take repartition flight visa not stamped,Want to go back to my family place. Currently staying in the hotel. I m stuck in India with my 8 month old infant. I can't take the repartition flights as my visa isn't stamped.
...,...,...,...,...
3713,4,4,economic downfall will create job crisis,Economic downfall will soon create job crises.
3714,4,4,employee not happy relief fund provided,Employee are not happy with the relief funds provided.
3715,4,4,should reduced le interest rate should charged loan other policy,emi should be reduced and less interest rate should be charged on loans and other policies.
3716,4,4,focus fake news,tv news channel focus more on fake news.


In [144]:
###################################################
X_train, X_test, y_train, y_test = train_test_split(df2, df2['label2'], test_size=0.2)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


# X_train.reset_index(drop=True)
# X_test.reset_index(drop=True)
# y_train.reset_index(drop=True)
# y_test.reset_index(drop=True)

In [145]:
###################################################
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

main=[]
for i in range(len(X_train)):
    arr=np.array([X_train['label1'][i],X_train['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_train)):
#     arr=np.array([X_train['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

# One-hot encode data
train_text = np.array(X_train, dtype=object)[:,2][:,np.newaxis]
# train_label =np.asarray(one_hot.fit_transform(main), dtype = np.int8)

train_label = np.zeros((X_train.shape[0], len(trainset.target)))
for i in range(X_train.shape[0]):
    train_label[i][main[i]] = 1

In [146]:
train_label[9], X_train.loc[9]

(array([1., 0., 0., 0., 0., 0.]),
 label1                                                                                                                                                                                                          0
 label2                                                                                                                                                                                                          0
 text                                                       experiencing poor connection one contacted network pathetic charging people ca not deliver service due poor network remote area unable to attend class
 text_     I am still experiencing a very poor connection! No one has contacted me. This network is pathetic! How are you still charging people when you can't deliver on your service? Due to poor network in ...
 Name: 9, dtype: object)

In [147]:
main[10],train_label[10], train_text[10]

(array([3, 5], dtype=int64),
 array([0., 0., 0., 1., 0., 1.]),
 array(['care medical wholesale distribution home health worker aide medical supply medical waste disposal hazardous waste disposal other ancillary healthcare service can go private 4 wheeler vehicle mother stuck'],
       dtype=object))

In [148]:
len(trainset.target)

6

In [149]:
###################################################
main=[]
for i in range(len(X_test)):
    arr=np.array([X_test['label1'][i],X_test['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_test)):
#     arr=np.array([X_test['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

test_text = np.array(X_test, dtype=object)[:,2][:,np.newaxis]
# test_label = np.asarray(one_hot.fit_transform(main), dtype = np.int8)

test_label = np.zeros((X_test.shape[0], len(trainset.target)))
for i in range(X_test.shape[0]):
    test_label[i][main[i]] = 1

In [172]:
###################################################
train_text_use = train_text[:, 0]
test_text_use = test_text[:, 0]
print (train_text_use)

['local auhtorities arranged raw vegetable to provide cooking oil spice to cook asking long nearby shop charging way price'
 '100000 crew member stranded cruise ship cruise line refuse to agree to pay repatriation expense hand sanitizer not need to wash water dry'
 'mother fallen sick admitted hospital doctor diagnosed might typhoid will to keep hospital observation adequate stockpile necessary medical supply type contingency known generation situation arise capitalism not safeguard public health'
 ...
 'water storage container bulb tubelights battery charger plug required warned hunger famine indication food shortage government push bogus agenda mean start preparing keep preparing'
 'tower area sparking issue canadian resident airport continue closed not humanitarian flight need to see family return to job waiting humanitarian flight early'
 'student worker stuck want to go home trying to livestream poor']


In [151]:
###################################################
category_counts = len(np.unique(df.label))
module_url = "../tf_sent_encoder_2" 
embed = hub.Module(module_url, trainable=True)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [152]:
###################################################
categories = trainset.target_names
categories

# from keras.layers import Concatenate

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = []
for c in categories:
    pred.append(Dense(1, activation='sigmoid')(dense2))
out = Concatenate(axis=-1)(pred)
model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [131]:
# def create_model(vocab_size, max_length):
#     model = Sequential()

#     model.add(Embedding(vocab_size, 128,
#             input_length = max_length,  trainable = False))
#     model.add(Bidirectional(GRU(128)))
#     model.add(Dense(64, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(BatchNormalization())
#     model.add(Dense(21, activation = "softmax"))

#     return model
print (K.eval(model.optimizer.lr))
K.set_value(model.optimizer.lr, 0.0005)
print (K.eval(model.optimizer.lr))

0.001
0.0005


In [41]:
# tf.initialize_all_variables().run(session=tf.keras.backend.get_session())
hub.__version__

'0.8.0'

In [40]:
###################################################
K.set_session(session)
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text_use, 
          train_label,
          validation_data=(test_text_use, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model20.h5')

Train on 2974 samples, validate on 744 samples
Epoch 1/1000


FailedPreconditionError: [_Derived_]{{function_node __inference_pruned_9755}} {{function_node __inference_pruned_9755}} Attempting to use uninitialized value Embeddings_en/sharded_1
	 [[{{node Embeddings_en/sharded_1/read}}]]
	 [[keras_layer/StatefulPartitionedCall]]

In [133]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 774       
Total params: 657,286
Trainable params: 657,286
Non-trainable params: 0
_________________________________________________________________


In [225]:
###################################################
model.load_weights('model15.h5')

In [154]:
###################################################
# def ElmoEmbeddingLayer(x):
# #     print(x.shape)
#     module = hub.Module("../nnlm-en-dim128_1", trainable=True)
#     embeddings = module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
#     return embeddings
nnlm_dim=128
nnlm_input = Input(shape=(), dtype=tf.string)
# nnlm_embedding = Lambda(ElmoEmbeddingLayer, output_shape=(nnlm_dim,))(nnlm_input)
nnlm_embedding = hub.KerasLayer("../nnlm-en-dim128_2", output_shape=[128], input_shape=[], dtype=tf.string, trainable=False)(nnlm_input)
x = Dense(512, activation='relu')(nnlm_embedding)
x = Dense(512, activation='relu')(x)
out = Dense(len(categories), activation='sigmoid')(x)

model_nnlm = Model(inputs=[nnlm_input], outputs=out)
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train, y_train, epochs=1,validation_data=(x_test, y_test))


LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model_nnlm.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [155]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           input_51[0][0]                   
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 1024)         525312      lambda_1[0][0]                   
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 128)          131200      dense_19[0][0]                   
___________________________________________________________________________________________

In [58]:

# K.set_session(session)
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text_use, 
          train_label,
          validation_data=(test_text_use, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model20.h5')

Train on 2974 samples, validate on 744 samples
Epoch 1/1000
2974/2974 [==============================] - 7s 2ms/step - loss: 0.5890 - acc: 0.6899 - val_loss: 0.5127 - val_acc: 0.7507
 - lr: 0.0010000000474974513
Epoch 2/1000
2974/2974 [==============================] - 1s 206us/step - loss: 0.4466 - acc: 0.8121 - val_loss: 0.3823 - val_acc: 0.8434
 - lr: 0.0010000000474974513
Epoch 3/1000
2974/2974 [==============================] - 1s 181us/step - loss: 0.3299 - acc: 0.8677 - val_loss: 0.3035 - val_acc: 0.8853
 - lr: 0.0010000000474974513
Epoch 4/1000
2974/2974 [==============================] - 1s 174us/step - loss: 0.2647 - acc: 0.8976 - val_loss: 0.2601 - val_acc: 0.8985
 - lr: 0.0010000000474974513
Epoch 5/1000
2974/2974 [==============================] - 0s 163us/step - loss: 0.2278 - acc: 0.9110 - val_loss: 0.2360 - val_acc: 0.9140
 - lr: 0.0010000000474974513
Epoch 6/1000
2974/2974 [==============================] - 1s 182us/step - loss: 0.2045 - acc: 0.9206 - val_loss: 0.2184 

2974/2974 [==============================] - 1s 275us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0693 - val_acc: 0.9778
 - lr: 0.0002500000118743628
Epoch 97/1000
2974/2974 [==============================] - 1s 262us/step - loss: 0.0027 - acc: 0.9994 - val_loss: 0.0695 - val_acc: 0.9776
 - lr: 0.0002500000118743628
Epoch 98/1000
2974/2974 [==============================] - 1s 267us/step - loss: 0.0027 - acc: 0.9994 - val_loss: 0.0693 - val_acc: 0.9785
 - lr: 0.0002500000118743628

Epoch 00098: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 99/1000
2974/2974 [==============================] - 1s 254us/step - loss: 0.0026 - acc: 0.9993 - val_loss: 0.0696 - val_acc: 0.9787
 - lr: 0.0001250000059371814
Epoch 100/1000
2974/2974 [==============================] - 1s 261us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0696 - val_acc: 0.9787
 - lr: 0.0001250000059371814
Epoch 101/1000
2974/2974 [==============================] - 1s 255us/step - loss: 0.0025 - a

In [224]:
###################################################
model_nnlm.load_weights('model_nnlm128_512_512.h5')

In [120]:
import csv
queries_list = []
with open('COVID-TEXT.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        queries_list.append(row[0])

In [121]:
queries_list

['Labourers without food',
 'Migrant Workers from UP and BIHAR',
 'People need cooked food',
 'people stuck in bangalore',
 'People Stuck without Food',
 'People without ration',
 'Residence of rama swamy palya facing water scarcity',
 'Stranded migrants from various states',
 '5o Migrant workers from West Bengal',
 'Alemari samudaya with food and ration',
 'Elder people need urgent ration',
 'Familiy without ration',
 '10 members need Ration , Daily Wage workers',
 '10 people Need information on how to go back to bihar ',
 '10 person from Bihar are struck in Dharwad. They need Ration , Kindly deliver, they have no money and food to survive .\nThanks',
 '14 Workers need ration',
 '150 people Wants to go home from mangalore to Giridih Jarkhand',
 '17 families are not having ration card not received any ration from anywhere. They need to be helped for the basic food and sanitaries',
 '25 people who are living in Venkateshwar Nagar, (Vaddargalli) not ration card and not benefiting from PD

In [158]:
for layer in model.layers:
    layer.trainable = False
    print (layer.name, layer.trainable)

input_51 False
lambda_1 False
dense_19 False
dense_20 False
dense_21 False
dense_22 False
dense_23 False
dense_24 False
dense_25 False
dense_26 False
concatenate_29 False


In [159]:
for layer in model_nnlm.layers:
    layer.trainable = False
    print (layer.name, layer.trainable)

input_52 False
keras_layer_1 False
dense_27 False
dense_28 False
dense_29 False


In [160]:
model_nnlm.output

<tf.Tensor 'dense_29/Sigmoid:0' shape=(?, 6) dtype=float32>

In [161]:

class Combine(Layer):
    def __init__(self, units=32, input_dim=32):
        super(Combine, self).__init__()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [187]:
###################################################

# model1 = np.array([[1.,2.,3.]])
# model2 = np.array([[5.,6.,7.]])
# input_text = Input(shape=(1,), dtype=tf.string)
# model1_out = model(input_text)
# input2 = Reshape(())(input_text)
# model2_out = model_nnlm(input2)
# comb = concatenate([model1_out, model2_out], axis=-1)
# out_l = Dense(1, use_bias=False)
# out = out_l(K.transpose(comb))
# out_l.get_weights()
# model_comb = Model(inputs=[input_text], outputs=out)

models = [model, model_nnlm]

input1 = Input(shape=(6), dtype=tf.float32)
input2 = Input(shape=(6), dtype=tf.float32)
reshaped1 = Reshape((6, 1, 1))(input1)
reshaped2 = Reshape((6, 1, 1))(input2)
concat = Concatenate(axis=2)([reshaped1, reshaped2])
conv = Conv2D(1, (1,2), padding='valid', use_bias=False)(concat)
reshaped = Reshape((6,))(conv)

model_comb = Model(inputs=[input1, input2], outputs=reshaped)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model_comb.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [173]:
###################################################
in1 = run_prediction_use(train_text_use)

['local auhtorities arranged raw vegetable to provide cooking oil spice to cook asking nearby shop charging way price', '100000 crew member stranded cruise ship cruise line refuse to agree to pay repatriation expense hand sanitizer not need to wash water dry', 'mother fallen sick admitted hospital doctor diagnosed might typhoid will to keep hospital observation adequate stockpile necessary medical supply type contingency known generation situation arise capitalism not safeguard public health', 'need medical assistance day student learning to bully school', "communication key pandemic charity need help making sure comms to scratch 's volunteer platform post opportunity request matched expert comms volunteer", 'reviewing health problem outbreak includes chronic disease injury influence health status same issue medical aid need written letter psychiatrist stating diagnosis chronic not know determined mine considered genetic', 'frequent power cut area five construction worker started walki

In [174]:
in1

array([[6.3878298e-04, 5.0738454e-04, 9.9818182e-01, 3.8743019e-07,
        9.9977827e-01, 1.1920929e-07],
       [2.3871660e-05, 5.5757791e-02, 1.3679266e-05, 9.9912858e-01,
        1.1920929e-07, 9.9999952e-01],
       [4.4703484e-07, 2.8938055e-05, 1.1729300e-03, 1.0000000e+00,
        6.2617958e-03, 2.8610229e-06],
       ...,
       [2.8036924e-02, 5.1980872e-02, 9.9969494e-01, 2.7574040e-02,
        8.0249265e-02, 5.1294905e-06],
       [9.8195535e-01, 1.1180213e-05, 6.2116118e-07, 3.2678086e-06,
        2.5397819e-03, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 1.2734988e-07, 3.5601549e-02,
        4.7606417e-08, 9.9388403e-01]], dtype=float32)

In [175]:
###################################################
in2 = run_prediction_nnlm(train_text_use)

['local auhtorities arranged raw vegetable to provide cooking oil spice to cook asking nearby shop charging way price', '100000 crew member stranded cruise ship cruise line refuse to agree to pay repatriation expense hand sanitizer not need to wash water dry', 'mother fallen sick admitted hospital doctor diagnosed might typhoid will to keep hospital observation adequate stockpile necessary medical supply type contingency known generation situation arise capitalism not safeguard public health', 'need medical assistance day student learning to bully school', "communication key pandemic charity need help making sure comms to scratch 's volunteer platform post opportunity request matched expert comms volunteer", 'reviewing health problem outbreak includes chronic disease injury influence health status same issue medical aid need written letter psychiatrist stating diagnosis chronic not know determined mine considered genetic', 'frequent power cut area five construction worker started walki

In [176]:
in2

array([[6.3878298e-04, 5.0738454e-04, 9.9818182e-01, 3.8743019e-07,
        9.9977827e-01, 1.1920929e-07],
       [2.3871660e-05, 5.5757791e-02, 1.3679266e-05, 9.9912858e-01,
        1.1920929e-07, 9.9999952e-01],
       [4.4703484e-07, 2.8938055e-05, 1.1729300e-03, 1.0000000e+00,
        6.2617958e-03, 2.8610229e-06],
       ...,
       [2.8036924e-02, 5.1980872e-02, 9.9969494e-01, 2.7574040e-02,
        8.0249265e-02, 5.1294905e-06],
       [9.8195535e-01, 1.1180213e-05, 6.2116118e-07, 3.2678086e-06,
        2.5397819e-03, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 1.2734988e-07, 3.5601549e-02,
        4.7606417e-08, 9.9388403e-01]], dtype=float32)

In [183]:
###################################################
in1_test = run_prediction_use(test_text_use)
in2_test = run_prediction_nnlm(test_text_use)

['5 people trapped internet locality passing stop group boy teased', '50 can sponsor 25 mask including shipping material sewists amount will appreciated electricity society electronic device phone wifi laptop shut', 'affected high risk developing must access food need to take dialysis', 'sister applied e pa to travel to beed local auhtorities arranged raw vegetable to provide cooking oil spice to cook asking', 'facing poor network slow speed issue idea domastic flight will resume', 'upcoming day usage will rise more petrol diesel food crisis lot food soaring price rotting crop coronavirus trigger global foodcrisis', 'due card issue unable to create hotspot shipping newborn clothes other essential nappy blanket lockdown', 'race to fill telecom not spot resourceful medical service use 3 g 2 g signal to overcome poor network coverage one taking care roadside animal starving plase to help', 'electricity electronic device phone wifi laptop shut tooth ache week need to see dentist', 'more gl

['5 people trapped internet locality passing stop group boy teased', '50 can sponsor 25 mask including shipping material sewists amount will appreciated electricity society electronic device phone wifi laptop shut', 'affected high risk developing must access food need to take dialysis', 'sister applied e pa to travel to beed local auhtorities arranged raw vegetable to provide cooking oil spice to cook asking', 'facing poor network slow speed issue idea domastic flight will resume', 'upcoming day usage will rise more petrol diesel food crisis lot food soaring price rotting crop coronavirus trigger global foodcrisis', 'due card issue unable to create hotspot shipping newborn clothes other essential nappy blanket lockdown', 'race to fill telecom not spot resourceful medical service use 3 g 2 g signal to overcome poor network coverage one taking care roadside animal starving plase to help', 'electricity electronic device phone wifi laptop shut tooth ache week need to see dentist', 'more gl

In [184]:
in2_test

array([[9.9999952e-01, 0.0000000e+00, 1.8477440e-05, 0.0000000e+00,
        9.9999905e-01, 5.4687262e-05],
       [9.9989289e-01, 1.6629696e-05, 0.0000000e+00, 9.9809796e-01,
        2.9143691e-04, 4.4428706e-03],
       [9.9566579e-04, 1.2847781e-04, 9.9930429e-01, 9.9999940e-01,
        0.0000000e+00, 3.3378601e-05],
       ...,
       [6.8843365e-06, 1.4084578e-04, 9.8674166e-01, 4.1484833e-05,
        4.2864829e-02, 9.9999881e-01],
       [9.9977154e-01, 9.9996161e-01, 2.2053719e-06, 3.6656857e-05,
        0.0000000e+00, 8.1455564e-01],
       [2.2351742e-06, 3.5230488e-02, 1.3715029e-04, 9.9946296e-01,
        1.1920929e-07, 9.9999988e-01]], dtype=float32)

In [87]:
model_comb.predict([np.array([[1,2,3,4,5,6]]), np.array([[1,2,3,4,5,6]])])


array([[-0.07140565, -0.1428113 , -0.21421719, -0.2856226 , -0.357028  ,
        -0.42843437]], dtype=float32)

In [276]:
###################################################
model_comb.layers[5].set_weights([np.array([[[[0.33]], [[0.67]]]])])

In [272]:
model_comb.layers[5].get_weights()

[array([[[[0.35]],
 
         [[0.65]]]], dtype=float32)]

In [98]:
session.run(model_comb.layers[5].weights[1])

array([0.], dtype=float32)

In [36]:
a = tf.Variable([1,2])
b = tf.Variable(3)
session.run(a.initializer)
session.run(b.initializer)
session.run(a*b)

array([3, 6])

In [231]:
###################################################
# K.set_session(session)
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model_comb.fit([in1, in2], 
          train_label,
          validation_data=([in1_test, in2_test], test_label),
          epochs=10,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model_comb.save_weights('./model_comb.h5')

Train on 2974 samples, validate on 744 samples
Epoch 1/10
2974/2974 [==============================] - 0s 30us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 2/10
2974/2974 [==============================] - 0s 24us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 3/10
2974/2974 [==============================] - 0s 21us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 4/10
2974/2974 [==============================] - 0s 28us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 5/10
2974/2974 [==============================] - 0s 31us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 6/10
2974/2974 [==============================] - 0s 32us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 7/10
2974/2974 [==============================] - 0s 33us/sample - loss: 0.0947 - acc: 0.9748 - val_loss: 0.0855 - val_acc: 0

In [266]:
###################################################
model_comb.save_weights('./model_comb.h5')

In [280]:
###################################################
def run_prediction_use(query_list):
    filt_query = [filter_query(q) for q in query_list]
    print (filt_query)
    # filt_query = query_list
    query_arr = np.array(filt_query, dtype=object)[:, np.newaxis]
    # query_arr = np.array(filt_query)
    predicts = model.predict(query_arr)
    return predicts

def run_prediction_nnlm(query_list):
    filt_query = [filter_query(q) for q in query_list]
    print (filt_query)
    # filt_query = query_list
    # query_arr = np.array(filt_query, dtype=object)[:, np.newaxis]
    query_arr = np.array(filt_query)
    predicts = model_nnlm.predict(query_arr)
    return predicts

def run_prediction_comb(query_list):
    filt_query = [filter_query(q) for q in query_list]
    print (filt_query)
    query_arr1 = np.array(filt_query, dtype=object)[:, np.newaxis]
    query_arr2 = np.array(filt_query)
    predicts1 = model.predict(query_arr1)
    predicts2 = model_nnlm.predict(query_arr2)
    print (predicts1, predicts2)
    pred = model_comb.predict([predicts1, predicts2])
    return pred

In [268]:
new_text = ["No network connectivity in my area for ordering groceries. Need ration urgently",
"Need gas cylinder as my mobile broke down.",
"My father is sick. i want daily ration",
"Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab to go to hospital.",
'I am unable to find fruits and vegetables',
'need food and travel for 50 families. contact number 8134981349',
'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance',
           'I want to travel back to my home. can you please provide glucose and emergency contact list for my journey.']


# new_text = np.array(new_text, dtype=object)[:, np.newaxis]
# #new_text = np.array(queries_list, dtype=object)[:, np.newaxis]

# predicts = model.predict(new_text)
predicts = run_prediction_comb(new_text)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

[[9.9999988e-01 1.0449737e-02 9.9128377e-01 2.0874143e-03 0.0000000e+00
  0.0000000e+00]
 [9.7125769e-04 9.9966490e-01 6.4641237e-05 9.8516750e-01 3.0398369e-06
  3.5762787e-06]
 [0.0000000e+00 5.6624413e-07 9.9999672e-01 9.9999988e-01 8.6847246e-03
  2.6822090e-07]
 [6.6757202e-05 0.0000000e+00 6.5544248e-04 1.2459606e-02 1.0000000e+00
  6.4671040e-06]
 [8.4769428e-03 1.6182661e-05 1.0000000e+00 9.2387199e-07 4.0538013e-03
  0.0000000e+00]
 [4.3795407e-03 5.9604645e-08 1.0000000e+00 5.8710575e-06 0.0000000e+00
  8.6782336e-01]
 [3.0994415e-06 5.8025122e-05 7.3206425e-04 1.0000000e+00 2.7952790e-03
  3.2782555e-07]
 [6.5892935e-05 1.3938546e-04 1.2212992e-04 7.5194216e-01 2.3841858e-07
  9.9999988e-01]] [[9.99995947e-01 1.36867166e-03 9.99999881e-01 0.00000000e+00
  1.01327896e-06 1.00731850e-05]
 [8.89897346e-05 1.00000000e+00 3.11970711e-04 3.15189362e-03
  3.49661708e-03 2.13536620e-03]
 [5.48362732e-06 1.05708838e-04 9.99999881e-01 9.07173336e-01
  2.38418579e-06 4.35656995e-01]
 [

In [234]:
new_text = ["No network connectivity in my area for order grocery . Need ration urgently",

 "i am try to contact airport authorities but they are not responding",

 "Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab as my car broke down to go to hospital.",

 'give me toilet cleaner and pet food',

 'ijust travel back to bangalore due to lockdown i am not able to find soap',

 'i want ventilators',

 'need to go home urgently.',

 'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance']
predicts = run_prediction_comb(new_text)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

[[1.0000000e+00 8.0019444e-02 9.9876457e-01 1.8209219e-05 0.0000000e+00
  0.0000000e+00]
 [9.9999988e-01 1.4936924e-03 2.2649765e-06 2.0861626e-07 1.1485815e-04
  9.9750674e-01]
 [1.9997358e-05 0.0000000e+00 1.2316406e-02 2.3186207e-05 1.0000000e+00
  3.4598476e-01]
 [2.0283461e-04 9.9999630e-01 8.8974166e-01 9.7990036e-05 8.7606907e-04
  8.9406967e-08]
 [4.8133391e-01 1.7283741e-01 0.0000000e+00 7.7486038e-07 4.6406358e-02
  9.9999946e-01]
 [0.0000000e+00 2.2791684e-02 1.2576580e-05 1.0000000e+00 9.6480203e-01
  0.0000000e+00]
 [1.4901161e-07 9.9851990e-01 1.3095140e-04 1.1253357e-04 0.0000000e+00
  9.3623346e-01]
 [3.0994415e-06 5.8025122e-05 7.3206425e-04 1.0000000e+00 2.7952790e-03
  3.2782555e-07]] [[9.9999940e-01 3.0538440e-04 1.0000000e+00 2.6822090e-07 8.9406967e-08
  1.1026859e-06]
 [1.0000000e+00 1.7198026e-03 1.0264218e-03 5.0306320e-05 8.5294247e-05
  1.2270883e-01]
 [0.0000000e+00 1.1920929e-07 2.9130578e-03 3.4557959e-01 9.9999988e-01
  9.5458418e-02]
 [5.9574842e-05 9.99

In [273]:
new_text = ['We are some helpless students stuck at hostel, where there is no mess facility availability due to absence of workers and no internet facility either, we need food and connectivity as early as possible',
           'My father has been unwell for some time now but the nearest hospital is almost 150 km away and we have no contact of any ambulance. Please make arrangements for an ambulance at the earliest. ',
           'I had started towards my home 120 km away on my bicycle but I am unable to continue further as I am weak and feverish. Please help me.',
           'I had started towards my home 120 km away on my bicycle but I am unable to continue further as I am weak and feverish. Please help me reach my home.',
           "I haven't seen my family for a long time and they need my support. They are lonely and helpless and I need to be there",
           'There is an unconscious lady at the Marathahalli bus stop and she has been lying there for over 4 hours. No helpline is responding but we need to get this heard to the authorities as soon as possible as she needs to get looked at.',
           "Even though the local party office is receiving supplies to feed our village, most of it is looted along the way. I have been trying to get this heard to the higher officials, but haven't been able to reach them for several days now. ",
           "There is a delivery man who has been delivering milk to several houses in our colony but yesterday I noticed him having red eyes and coughing, I know I need to report this to someone but don't know what to do",
           "I came to a small village to perform a government inspection for a few days but after the lockdown announcement I haven't been able to return. I feel lonely and unsafe here, please help me. ",
           ]

predicts = run_prediction_comb(new_text)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

[[9.8691064e-01 0.0000000e+00 9.9998891e-01 2.9802322e-08 1.6415119e-04
  8.8098752e-01]
 [9.6257597e-01 5.9604645e-08 0.0000000e+00 1.0000000e+00 3.6954880e-06
  9.0200788e-01]
 [2.3186207e-05 0.0000000e+00 8.9734793e-05 9.9282372e-01 9.5789480e-01
  9.9772871e-01]
 [2.1666288e-05 0.0000000e+00 2.1994114e-05 7.2701204e-01 9.2596871e-01
  9.9997079e-01]
 [4.3542960e-01 0.0000000e+00 1.0073185e-05 0.0000000e+00 1.0079107e-01
  1.0000000e+00]
 [1.0000000e+00 7.6919794e-05 2.2053719e-06 9.9997985e-01 3.8743019e-07
  0.0000000e+00]
 [1.3546050e-03 9.7934377e-01 1.2874514e-02 3.4719706e-05 9.5281422e-01
  1.8528104e-04]
 [2.1552294e-02 1.3538271e-02 1.8859237e-02 9.9958718e-01 9.9324256e-01
  0.0000000e+00]
 [5.3692338e-06 9.0826255e-08 7.0261436e-08 1.4463659e-06 9.9999976e-01
  1.0000000e+00]] [[9.9920976e-01 5.9604645e-08 2.5588870e-03 7.0087051e-01 1.1920929e-06
  9.9986875e-01]
 [9.3430281e-05 0.0000000e+00 5.9008598e-06 1.0000000e+00 1.7583370e-06
  7.7077806e-02]
 [2.3402661e-02 2.59

In [56]:
model.predict(np.array(['need to go home', ''], dtype=object)[:, np.newaxis])

array([[6.0719925e-05, 5.1792540e-02, 3.0541523e-05, 3.5929384e-05,
        7.5689690e-07, 3.2593703e-01],
       [8.1119651e-05, 9.8318785e-01, 2.1166863e-06, 1.6125040e-02,
        6.0552239e-02, 5.8824737e-03]], dtype=float32)

In [107]:
filter_query("i want roti ,  egg , dal , chawal , anda .")

[('i', 'PRON'), ('want', 'VERB'), ('roti', 'NOUN'), (',', 'PUNCT'), ('egg', 'NOUN'), (',', 'PUNCT'), ('dal', 'PROPN'), (',', 'PUNCT'), ('chawal', 'PROPN'), (',', 'PUNCT'), ('anda', 'PROPN'), ('.', 'PUNCT')]


'want roti egg'

In [39]:
result_df = pd.DataFrame([[queries_list[i]] + list(np.around(predicts*100,decimals=1)[i]) for i in range(len(queries_list))], columns=['query'] + dictt)

IndexError: index 7 is out of bounds for axis 0 with size 7

In [50]:
result_df.head()

,query,communication,food,healthcare,others,travel
0,Labourers without food,0.000000,100.0,0.000000,0.800000,0.000000
1,Migrant Workers from UP and BIHAR,0.000000,0.6,48.900002,0.100000,99.099998
2,People need cooked food,0.300000,100.0,0.200000,64.699997,0.000000
3,people stuck in bangalore,98.099998,0.0,0.000000,100.000000,0.000000
4,People Stuck without Food,14.600000,100.0,0.000000,8.600000,0.000000


In [51]:
result_df.to_csv('results.csv', index=False)

In [24]:
query_arr = np.array(['need to visit home', ''], dtype=object)[:, np.newaxis]
model.predict(query_arr)

array([[2.2256071e-07, 1.9670317e-04, 2.2590014e-03, 4.9715425e-04,
        9.9989069e-01],
       [3.3535997e-03, 5.9001707e-03, 6.5540969e-01, 1.1426200e-02,
        2.5253209e-01]], dtype=float32)

In [275]:
new_text = ["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them. Please provide ration for all of us.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help",
'We are a small village on the outskirts of Bangalore and we do not have good cell reception here and hence we are isolated here with no news about coronavirus or availability of supplies.',
'Three construction workers from jharkhand are hungry from last two days and jumped off the building. Please provide ambulance',
'my friend fell from terrace while searching for network. There is network problem in this area. This is all government mistake and the medical expenses should be bourne by them.',
'The daily ration kit distributed to us do not have cooking oil in them. How are we supposed to cook.',
"I am trying to start a small kitchen to feed the hungry poor kids in my locality but I am unable to do so as I haven't received the big 19 kg gas cannister which I applied for almost 20 days ago.",
'I am a doctor, my car brokedown and want it to get repaired urgently.']
# new_text = np.array(new_text, dtype=object)[:, np.newaxis]


# predicts = model.predict(new_text)


# categories = np.unique(df.label)
# predict_logits = predicts.argmax(axis=1)
# predict_labels = [categories[logit] for logit in predict_logits]
predicts = run_prediction_comb(new_text)

dictt=trainset.target_names
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

[[7.1525574e-07 5.6624413e-07 1.0000000e+00 4.0531158e-06 7.1525574e-07
  1.1509657e-04]
 [2.9246360e-02 0.0000000e+00 9.9999630e-01 0.0000000e+00 8.3817077e-01
  9.9791044e-01]
 [1.7166138e-05 1.1920929e-07 9.9995744e-01 1.5154481e-04 8.7320805e-06
  9.9873441e-01]
 [4.6491623e-06 5.5592310e-01 9.9620527e-01 4.0907225e-01 5.8200455e-01
  3.8444996e-06]
 [8.3054066e-02 0.0000000e+00 7.2421944e-01 1.1327773e-02 9.9593210e-01
  1.0039210e-03]
 [1.5637577e-02 0.0000000e+00 9.9999994e-01 1.7881393e-07 9.1720819e-02
  6.5585375e-02]
 [9.9999988e-01 1.0132790e-06 9.9999946e-01 8.0466270e-07 0.0000000e+00
  5.9372187e-04]
 [1.0000000e+00 0.0000000e+00 3.1280518e-04 3.6358833e-06 1.5527010e-05
  2.2158027e-03]
 [1.3421664e-05 6.0600589e-05 1.6431359e-08 9.9490052e-01 7.8499043e-01
  9.9074090e-01]
 [9.9999273e-01 0.0000000e+00 0.0000000e+00 9.8849070e-01 9.9984670e-01
  9.3713179e-06]
 [6.8977010e-07 0.0000000e+00 1.0000000e+00 3.2292658e-03 6.2824838e-02
  1.5272421e-03]
 [2.1595932e-03 2.624

In [281]:
new_text = ["brush toothpaste and soap for 2 people and 50 units of n95 mask",
            "brush toothpaste and soap and 50 units of n95 mask",
           "rice, wheat and daal for 2 people and 50 units of n95 mask and syrup",
           "And the second is: Sri Venkateswara Layout, Chinappa Layout, Mahadevapura. There are 350 workers- construction, recycling, domestic help. WB, UP, Bihar. need urgent dry ration",
           "There are 350 workers- construction, recycling, domestic help. WB, UP, Bihar. need urgent dry ration",
           "et rations from a govt distribution in Sunkadkatte they didn't give him saying he is not from here. So request help here in this case too since they are not locals",
           '''Respected Ma'am,
Help Required.
No ration since yesterday who are in really need of food .
Pa Shehgin & family including 2 children.
1. Pa. Shehgin kipgen( worked in sales )
2. Nu. Hoineng kipgen( worked beauty salon )
3. Chingnu( daughter )
4. Dedai( daughter )
5. Gugun( son)
6. Hekham
7. Hoineo
8. Chinkho
Address:
#12, 19 A cross Lakshmipuram, Halasuru, Bengaluru 560008.
Contact number : 8884356927
1.Mrs.hahat doungel ( working in beauty salon) 2.boichung doungel ( son,7 year old )
3.jenny.doungel ( sister ).
Contact no : 9108396170.
Address: Kothanur, Anjinapalia, 3 cross''',
           "intrested in donating used blankets",
           "intrested in used blankets"
]
# new_text = np.array(new_text, dtype=object)[:, np.newaxis]


# predicts = model.predict(new_text)


# categories = np.unique(df.label)
# predict_logits = predicts.argmax(axis=1)
# predict_labels = [categories[logit] for logit in predict_logits]
predicts = run_prediction_comb(new_text)

dictt=trainset.target_names
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['brush toothpaste soap people unit mask', 'brush toothpaste soap unit mask', 'rice wheat daal people unit mask syrup', 'second layout workers- construction recycling domestic help need urgent dry ration', 'workers- construction recycling domestic help need urgent dry ration', 'ration govt distribution sunkadkatte not give saying not request help case not local', 'respected help required ration yesterday need food family including child worked sale worked beauty salon daughter daughter address cross contact number working beauty salon year old sister contact address cross', 'intrested in donating used blanket', 'intrested in used blanket']
[[0.00000000e+00 2.19443232e-01 3.42667103e-04 7.21921444e-01
  9.99748230e-01 8.06101859e-02]
 [0.00000000e+00 7.13512421e-01 4.20749187e-04 4.55765724e-01
  8.54732275e-01 7.96801865e-01]
 [2.98023224e-08 0.00000000e+00 9.99999285e-01 2.50908732e-03
  8.44455361e-02 1.33395195e-04]
 [1.77323818e-05 2.20030546e-04 1.86415851e-01 1.95649981e-01
  1.0

In [ ]:
["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help"]

In [33]:
predicts.shape
preds_round = np.around(predicts*100,decimals=1)

In [69]:
from statistics import stdev
pred_results = []
for i in range(predicts.shape[0]):
    sorted_class_similariy = [float(x) for x in sorted(preds_round[i], reverse=True)]
    print (sorted_class_similariy)
    split_stdev = []
    split_stdev.append(stdev(sorted_class_similariy[1:len(dictt)]))
    for k in range(2, len(dictt) - 1):
        split_stdev.append((stdev(sorted_class_similariy[:k]) + stdev(sorted_class_similariy[k:]))/2)
    split_stdev.append(stdev(sorted_class_similariy[:len(dictt) - 1]))

    split_ind = np.argmin(split_stdev)
    thres = (sorted_class_similariy[split_ind] + sorted_class_similariy[split_ind + 1]) / 2
    # print (thres)
    pred_labels = [c for c in range(len(dictt)) if preds_round[i][c] >= np.around(thres, decimals=1)]
    pred_classes = [dictt[r] for r in pred_labels]
    # print (pred_labels)
    pred_results.append(pred_classes)

[100.0, 92.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 6.800000190734863, 5.5, 0.20000000298023224, 0.0, 0.0]
[99.9000015258789, 83.80000305175781, 56.900001525878906, 0.0, 0.0, 0.0]
[94.80000305175781, 93.80000305175781, 35.099998474121094, 2.200000047683716, 0.8999999761581421, 0.0]
[100.0, 70.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 71.9000015258789, 0.0, 0.0, 0.0, 0.0]
[97.80000305175781, 41.79999923706055, 20.0, 10.0, 0.0, 0.0]
[100.0, 99.9000015258789, 3.200000047683716, 0.30000001192092896, 0.10000000149011612, 0.0]
[100.0, 4.199999809265137, 2.4000000953674316, 0.6000000238418579, 0.20000000298023224, 0.0]
[100.0, 100.0, 0.10000000149011612, 0.10000000149011612, 0.0, 0.0]
[100.0, 51.099998474121094, 2.299999952316284, 0.0, 0.0, 0.0]
[100.0, 27.5, 0.20000000298023224, 0.0, 0.0, 0.0]
[100.0, 2.200000047683716, 0.4000000059604645, 0.4000000059604645, 0.20000000298023224, 0.0]
[99.9000015258789, 6.5, 3.4000000953674316, 0.5, 0.20000000298023224, 0.10000000149011612]
[99.8000030517

In [70]:
pred_results

[['food', 'health'],
 ['food'],
 ['food', 'fuel', 'health'],
 ['communication', 'travel'],
 ['food', 'health'],
 ['food', 'health'],
 ['travel'],
 ['food', 'travel'],
 ['food'],
 ['food', 'health'],
 ['abuse', 'food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['communication', 'food'],
 ['food', 'travel'],
 ['food', 'health', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['food', 'health', 'travel'],
 ['food', 'fuel', 'travel'],
 ['health'],
 ['food'],
 ['travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['communication', 'travel'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food', 'fuel', 'health'],
 ['food', 'fuel', 'travel'],
 ['fuel'],
 ['food', 'fuel', 'travel'],
 ['food'],
 ['food'],
 ['abuse'],
 ['food', 'fuel'],
 ['travel'],
 ['food'],
 ['food', 'travel'],
 ['abuse', 'food'],
 ['abuse', 'fuel', 

In [ ]:
OK### both tanh -84% split=0.3    split=o.2 good
## both relu - 86% split=0.2
## both tanh - 85.4% split=0.2
## tanh relu 84.73
### relu tanh 85.63 -----good 256,128
### relu tanh  86.12-----ok ok 1024,256
### relu tanh  86.12-----ok ok 1024,256
## tanh tanh ---1024 128 
## relu tanh tanh 256 128 64- 85.8%
## tanh tanh 0.3 split acc= 84.55 can try


OK###with categorical cross entropy tanh tanh 256 128 results are ok but accuracy showing is less




### 93.82%
# dense1 = Dense(256, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])




### 94.41%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(256, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



## __94.78____%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



####### 94.48 % #########

# input_text = Input(shape=(1,), dtype=tf.string)
# embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='tanh')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [161]:

# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

In [162]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [157]:
# elmo = hub.Module('../elmo_2', trainable=True)
# elmo(K.squeeze(K.cast([['food'],['']], tf.string), axis=1),
#                         as_dict=True,
#                         signature='default',
#                         )['default']

In [140]:
# model_url = "https://tfhub.dev/google/elmo/3"
# # hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
# hub_layer = hub.KerasLayer(model_url, output_shape=[128], input_shape=[], 
#                            dtype=tf.string, trainable=False, 
#                            signature="default", output_key="default")
# hub_layer(train_examples[:3])
elmo_emb_layer = ElmoEmbeddingLayer().build((None,))
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(len(categories), activation='sigmoid'))
model.build(input_shape=())
model.summary()



# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# out = Dense(len(categories), activation='sigmoid')(dense2)
# # pred = []
# # for c in categories:
# #     pred.append(Dense(1, activation='sigmoid')(dense2))
# # out = Concatenate()(pred)
# model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


NameError: name 's' is not defined

In [ ]:
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(6, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

model = build_model()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=10,
          batch_size=32)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_18 (Elm (None, 1024)              4         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 1542      
Total params: 263,946
Trainable params: 263,946
Non-trainable params: 0
_________________________________________________________________
Train on 2974 samples, validate on 744 samples
Epoch 1/10


In [26]:

# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [27]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 512
        self.trainable=False
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('../tf_sent_encoder_2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        # self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [29]:
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(6, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

model = build_model()

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=10,
          batch_size=32)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_2 (Elmo (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1542      
Total params: 132,870
Trainable params: 132,870
Non-trainable params: 0
_________________________________________________________________
Train on 2974 samples, validate on 744 samples
Epoch 1/10
2974/2974 [==============================] - 11s 4ms/step - loss: 0.4565 - acc: 0.8005 - val_loss: 0.3268 - val_acc: 0.87